In [33]:
import os
from utils import *
import keras.layers.convolutional as convolutional
from keras.models import Sequential
import json
%matplotlib inline
from __future__ import print_function, division
import numpy
from os import listdir
import pandas as pd
from shutil import copyfile, rmtree
from IPython.display import FileLink
# reload(vgg_statefarm)
# from vgg_statefarm import Vgg16
import vgg_statefarm as v
from vgg_statefarm import BcolzArrayIterator

In [32]:
reload(vgg_statefarm)

<module 'vgg_statefarm' from 'vgg_statefarm.py'>

## Data Set Up


The best approach would be to create a bunch of models, each time holding out one driver, and then average the validation across all of them (you could also average the predictions across all of them, like the ensembling we did in MNIST last week) But I wouldn't bother with that until you had done all the experimenting you wanted to do, since it adds a lot of time to each experiment.

In [21]:
# v.mn_valid(local=True)
# v.rm_sample_set(local=True)
# v.mn_sample(local=True)

In [22]:
current_dir = os.getcwd()
data_dir = current_dir + '/data/statefarm/'
# data_dir = current_dir + '/data/statefarm/sample/'
results_path=data_dir + 'results/'
train_path=data_dir + 'train/'
valid_path=data_dir + 'valid/'
test_path=data_dir + 'test/'
model_path=data_dir + 'models/'

## Peel VGG back to Convolutions, Data Aug, Normalization

In [23]:
vgg = v.Vgg16()
vgg.ft(num=10)

In [38]:
batch_size=128
gen_t = image.ImageDataGenerator(rotation_range=15, height_shift_range=0.05, 
                shear_range=0.1, channel_shift_range=20, width_shift_range=0.1)
aug_trn_batches = get_batches(train_path, gen_t, batch_size=batch_size)
aug_val_batches = get_batches(valid_path, batch_size=batch_size*2, shuffle=False)
trn_batches = get_batches(train_path, batch_size=batch_size)
val_batches = get_batches(valid_path, batch_size=batch_size*2, shuffle=False)
test_batches = get_batches(test_path, batch_size=batch_size*2, shuffle=False)

Found 15671 images belonging to 10 classes.
Found 6753 images belonging to 10 classes.
Found 15671 images belonging to 10 classes.
Found 6753 images belonging to 10 classes.
Found 79726 images belonging to 1 classes.


In [6]:
aug_trn_features = vgg.model.predict_generator(aug_trn_batches, aug_trn_batches.nb_sample*5)

In [7]:
save_array(model_path+'aug_trn_features.dat', aug_trn_features)

In [8]:
trn_features = vgg.model.predict_generator(trn_batches, trn_batches.nb_sample)

In [9]:
save_array(model_path+'trn_features.dat', trn_features)

In [8]:
val_features = vgg.model.predict_generator(val_batches, val_batches.nb_sample)

In [9]:
save_array(model_path+'val_features.dat', val_features)

In [7]:
test_features = vgg.model.predict_generator(test_batches, test_batches.nb_sample)

In [8]:
save_array(model_path+'test_features.dat', test_features)

In [ ]:
all_trn_features = np.concatenate([aug_trn_features, trn_features])

In [ ]:
save_array(model_path+'all_trn_features.dat', all_trn_features)

In [39]:
aug_trn_features = bcolz.open(model_path+'aug_trn_features.dat')
trn_features = bcolz.open(model_path+'trn_features.dat')
val_features = bcolz.open(model_path+'val_features.dat')
test_features = bcolz.open(model_path+'test_features.dat')


In [40]:
# aug_trn_features = load_array(model_path+'aug_trn_features.dat')
# trn_features = load_array(model_path+'trn_features.dat')
# all_trn_features = load_array(model_path+'all_trn_features.dat')
# val_features = load_array(model_path+'val_features.dat')
# test_features = load_array(model_path+'test_features.dat')

In [41]:
trn_labels = onehot(trn_batches.classes)
val_labels = onehot(val_batches.classes)
aug_trn_labels = np.concatenate([trn_labels]*5)
all_trn_labels = np.concatenate([trn_labels]*6)

In [42]:
aug_trn_batches_bcolz = BcolzArrayIterator(aug_trn_features, aug_trn_labels, batch_size=aug_trn_features.chunklen * 10, shuffle=True)
trn_batches_bcolz = BcolzArrayIterator(trn_features, trn_labels, batch_size=trn_features.chunklen * 10, shuffle=True)
val_batches_bcolz = BcolzArrayIterator(val_features, val_labels, batch_size=val_features.chunklen * 20, shuffle=False)
# test_batches = BcolzArrayIterator(test_features, test_labels, batch_size=trn_features.chunklen * 20, shuffle=False)

In [57]:
aug_model = vgg.get_fc_model(.2, aug_trn_batches)
aug_model.compile(Adam(lr=1e-10), loss='categorical_crossentropy', metrics=['accuracy'])

In [58]:
batch_size=64
no_of_epoch=2

In [59]:
trn_batches.nb_sample
trn_batches_bcolz.N
val_batches_bcolz.N
val_batches.nb_sample

6753

In [60]:
# aug_model.fit(aug_trn_features, aug_trn_labels, batch_size=batch_size, nb_epoch=no_of_epoch, 
#              validation_data=(val_features, val_labels))

aug_model.fit_generator(trn_batches_bcolz, samples_per_epoch=trn_batches_bcolz.N, nb_epoch=no_of_epoch, validation_data=val_batches_bcolz, nb_val_samples=val_batches_bcolz.N)

# aug_model.fit(all_trn_features, all_trn_labels, batch_size=batch_size, nb_epoch=no_of_epoch, 
#              validation_data=(val_features, val_labels))

Epoch 1/2
15671/15671 [==============================] - 12s - loss: 3.0491 - acc: 0.1027 - val_loss: 2.7064 - val_acc: 0.1060
Epoch 2/2
15671/15671 [==============================] - 12s - loss: 3.0411 - acc: 0.1022 - val_loss: 2.7270 - val_acc: 0.1056


In [ ]:
aug_model.optimizer.lr=0.01
no_of_epoch=4

In [ ]:
aug_model.fit(all_trn_features, all_trn_labels, batch_size=batch_size, nb_epoch=no_of_epoch, 
             validation_data=(val_features, val_labels))

In [ ]:
bn_model.optimizer.lr=0.0001
no_of_epoch=4

In [ ]:
aug_model.fit(all_trn_features, all_trn_labels, batch_size=batch_size, nb_epoch=no_of_epoch, 
             validation_data=(val_features, val_labels))

In [ ]:
aug_model.save_weights(model_path+'aug_model_weights.h5')

## Submit

In [64]:
def do_clip(arr, mx): return np.clip(arr, (1-mx)/9, mx)

In [ ]:
test_features = load_array(results_path+'test_features.dat')

In [ ]:
val_preds = aug_model.predict(val_features, batch_size=batch_size*2)
preds = aug_model.predict(test_features, batch_size=batch_size*2)

In [ ]:
keras.metrics.categorical_crossentropy(val_labels, do_clip(val_preds, 0.93)).eval()

In [ ]:
subm = do_clip(preds,0.93)

In [ ]:
subm_name = results_path+'/subm.gz'

In [ ]:
classes = sorted(batches.class_indices, key=batches.class_indices.get)

In [ ]:
submission = pd.DataFrame(subm, columns=classes)
submission.insert(0, 'img', [a[4:] for a in test_filenames])
submission.head()

In [ ]:
submission.to_csv(subm_name, index=False, compression='gzip')

In [ ]:
FileLink(subm_name)

## Scratchwork

In [54]:
def proc_wgts(layer, prev_p, new_p):
    scal = (1-prev_p)/(1-new_p)
    return [o*scal for o in layer.get_weights()]

opt = RMSprop(lr=0.00001, rho=0.7)
def get_fc_model(p, batches):
    model = Sequential([
        MaxPooling2D(input_shape=vgg.model.layers[-1].output_shape[1:]),
        Flatten(),
        Dense(256, activation='relu'),
        Dropout(p),
        BatchNormalization(),
        Dense(256, activation='relu'),
        Dropout(p),
        BatchNormalization(),
        Dense(batches.nb_class, activation='softmax')
        ])

    for l in model.layers: 
        if type(l)==Dense: l.set_weights(proc_wgts(l, 0.3, 0.6))

    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [56]:
fc_model = get_fc_model(batches)

In [58]:
batch_size=8
no_of_epochs=2

In [61]:
fc_model.fit(trn_features, trn_labels, nb_epoch=no_of_epochs, batch_size=batch_size, validation_data=(val_features, val_labels))

Train on 1500 samples, validate on 1000 samples
Epoch 1/2
1500/1500 [==============================] - 184s - loss: 4.2066 - acc: 0.1107 - val_loss: 4.6126 - val_acc: 0.0950
Epoch 2/2
1500/1500 [==============================] - 176s - loss: 2.2797 - acc: 0.3360 - val_loss: 4.3633 - val_acc: 0.0860


In [62]:
fc_model.optimizer.lr = 0.00001
fc_model.fit(trn_features, trn_labels, nb_epoch=no_of_epochs, batch_size=batch_size, validation_data=(val_features, val_labels))

Train on 1500 samples, validate on 1000 samples
Epoch 1/2
1500/1500 [==============================] - 182s - loss: 1.1495 - acc: 0.6080 - val_loss: 4.3291 - val_acc: 0.0960
Epoch 2/2
1500/1500 [==============================] - 176s - loss: 0.5279 - acc: 0.8360 - val_loss: 4.3173 - val_acc: 0.1000


In [ ]:
no_of_epochs=4
fc_model.optimizer.lr = 0.001
fc_model.fit(trn_features, trn_labels, nb_epoch=no_of_epochs, batch_size=batch_size, validation_data=(val_features, val_labels))

In [ ]:
fc_model.save_weights(model_path+'sf_sample.h5')
fc_model.load_weights(model_path+'sf_sample.h5')

## Create Batches

In [142]:
def conv1(batches):
    model = Sequential([
            BatchNormalization(axis=1, input_shape=(3,224,224)),
            Convolution2D(32,3,3, activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
            Convolution2D(64,3,3, activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
            Flatten(),
            Dense(200, activation='relu'),
            BatchNormalization(),
            Dense(10, activation='softmax')
        ])

    model.compile(Adam(lr=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit_generator(batches, batches.nb_sample, nb_epoch=2, validation_data=val_batches, 
                     nb_val_samples=val_batches.nb_sample)
    model.optimizer.lr = 0.001
    model.fit_generator(batches, batches.nb_sample, nb_epoch=4, validation_data=val_batches, 
                     nb_val_samples=val_batches.nb_sample)
    return model

def conv2(batches):
    model = Sequential([
        BatchNormalization(axis=1, input_shape=(3,224,224)),
        Convolution2D(32,3,3, activation='relu'),
        BatchNormalization(axis=1),
        MaxPooling2D(),
        Convolution2D(64,3,3, activation='relu'),
        BatchNormalization(axis=1),
        MaxPooling2D(),
        Convolution2D(128,3,3, activation='relu'),
        BatchNormalization(axis=1),
        MaxPooling2D(),
        Flatten(),
        Dense(200, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(200, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(10, activation='softmax')
    ])

In [7]:
gen_t = image.ImageDataGenerator(rotation_range=15, height_shift_range=0.05, 
                shear_range=0.1, channel_shift_range=20, width_shift_range=0.1)
batches = get_batches(train_path, gen_t, batch_size=batch_size)
val_batches = get_batches(valid_path, batch_size=batch_size*2, shuffle=False)

Found 1500 images belonging to 10 classes.
Found 1000 images belonging to 10 classes.


In [10]:
(val_classes, trn_classes, val_labels, trn_labels, val_filenames, filenames, test_filenames) = get_classes(data_dir)

Found 15671 images belonging to 10 classes.
Found 6753 images belonging to 10 classes.
Found 0 images belonging to 0 classes.


In [13]:
trn = get_data(train_path)
val = get_data(valid_path)

Found 1500 images belonging to 10 classes.
Found 1000 images belonging to 10 classes.


In [16]:
save_array(data_dir+'results/val.dat', val)
save_array(data_dir+'results/trn.dat', trn)
# val = load_array(path+'results/val.dat')
# trn = load_array(path+'results/trn.dat')

In [17]:
type(trn)

numpy.ndarray

In [146]:
model = conv1(batches)

Epoch 1/2
1500/1500 [==============================] - 33s - loss: 2.2838 - acc: 0.2693 - val_loss: 2.1581 - val_acc: 0.3030
Epoch 2/2
1500/1500 [==============================] - 33s - loss: 1.7052 - acc: 0.4447 - val_loss: 1.6523 - val_acc: 0.4790
Epoch 1/4
1500/1500 [==============================] - 34s - loss: 1.5338 - acc: 0.4887 - val_loss: 1.6045 - val_acc: 0.4600
Epoch 2/4
1500/1500 [==============================] - 32s - loss: 1.4162 - acc: 0.5093 - val_loss: 1.4913 - val_acc: 0.5350
Epoch 3/4
1500/1500 [==============================] - 33s - loss: 1.3188 - acc: 0.5720 - val_loss: 1.4043 - val_acc: 0.5080
Epoch 4/4
1500/1500 [==============================] - 33s - loss: 1.2640 - acc: 0.5867 - val_loss: 1.2721 - val_acc: 0.5950


In [147]:
model.optimizer.lr = 0.0001
model.fit_generator(batches, batches.nb_sample, nb_epoch=5, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)

Epoch 1/5
1500/1500 [==============================] - 33s - loss: 1.1632 - acc: 0.6200 - val_loss: 1.4224 - val_acc: 0.5550
Epoch 2/5
1500/1500 [==============================] - 32s - loss: 1.0810 - acc: 0.6553 - val_loss: 1.3246 - val_acc: 0.5780
Epoch 3/5
1500/1500 [==============================] - 33s - loss: 1.0340 - acc: 0.6740 - val_loss: 1.2650 - val_acc: 0.6030
Epoch 4/5
1500/1500 [==============================] - 32s - loss: 1.0163 - acc: 0.6667 - val_loss: 1.0990 - val_acc: 0.6510
Epoch 5/5
1500/1500 [==============================] - 32s - loss: 1.0364 - acc: 0.6600 - val_loss: 1.4614 - val_acc: 0.5510


In [148]:
model.fit_generator(batches, batches.nb_sample, nb_epoch=25, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)

Epoch 1/25
1500/1500 [==============================] - 34s - loss: 0.9446 - acc: 0.7000 - val_loss: 1.0697 - val_acc: 0.6660
Epoch 2/25
1500/1500 [==============================] - 32s - loss: 0.9120 - acc: 0.7287 - val_loss: 1.1718 - val_acc: 0.6330
Epoch 3/25
1500/1500 [==============================] - 33s - loss: 0.9165 - acc: 0.7113 - val_loss: 1.1122 - val_acc: 0.6380
Epoch 4/25
1500/1500 [==============================] - 32s - loss: 0.8506 - acc: 0.7373 - val_loss: 1.1121 - val_acc: 0.6100
Epoch 5/25
1500/1500 [==============================] - 32s - loss: 0.7845 - acc: 0.7553 - val_loss: 0.9349 - val_acc: 0.6820
Epoch 6/25
1500/1500 [==============================] - 33s - loss: 0.7507 - acc: 0.7640 - val_loss: 1.0202 - val_acc: 0.6560
Epoch 7/25
1500/1500 [==============================] - 33s - loss: 0.7962 - acc: 0.7540 - val_loss: 0.9893 - val_acc: 0.6760
Epoch 8/25
1500/1500 [==============================] - 33s - loss: 0.7403 - acc: 0.7647 - val_loss: 0.9372 - val_acc:

KeyboardInterrupt: 